In [1]:
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm
from sklearn import datasets

In [2]:
data = datasets.load_boston()
print(data)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.DataFrame(data.data,columns=data.feature_names)
target = pd.DataFrame(data.target, columns=["MEDV"])

In [5]:
print(df.head(3))
print(target.head(3))
print(df.info())

In [6]:
## without constant 
X = df
y = target

In [7]:
#variable selection using RF

In [8]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFECV(estimator, cv=10,scoring='mean_squared_error',n_jobs=10)
selector.fit(X, y)
print('Optimal number of features: %d' % selector.n_features_)
print (X.columns[selector.support_])

In [9]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score MSE")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

In [10]:
#Stats model


In [11]:
x = df[['CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT']]
y = target

In [12]:
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4,random_state=1)

In [13]:
model = sm.OLS(y_train,X_train).fit()

In [14]:
predication = model.predict(X_train)

In [15]:
model.summary()
# to understand read this https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

In [16]:
X_train = sm.add_constant(X_train) ## let's add an intercept (beta_0) to our model
model = sm.OLS(y_train,X_train).fit()
#    or
#Dimport statsmodels.formula.api as smf
#model =  smf.ols('y ~ x1 * x2 * x3', df).fit()
model.summary()
predication_train = model.predict(X_train)

In [17]:
plt.figure(1, figsize = (15,7))
plt.scatter(y_train, predication_train)
plt.ylabel('Predication')
plt.xlabel('Training')
plt.grid(True)
plt.show()

In [18]:
# Testing on test data

In [19]:
X_test = sm.add_constant(X_test) ## let's add an intercept (beta_0) to our model
predication_test = model.predict(X_test)

In [20]:
plt.figure(1, figsize = (15,7))
plt.scatter(y_test, predication_test)
plt.ylabel('Predication')
plt.xlabel('Training')
plt.grid(True)
plt.show()

In [21]:
#Sklean model

In [22]:
from sklearn import linear_model

In [23]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
predication_train = model.predict(X_train)
print(predication_train[0:5],y_train[0:5])

In [24]:
lm.score(X_train,y_train) # This is the R² score of our model. 

In [25]:
lm.coef_

In [26]:
lm.intercept_

In [27]:
import matplotlib.pyplot as plt

In [28]:
plt.figure(1, figsize = (15,7))
plt.scatter(y_train, predication_train)
plt.ylabel('Predication')
plt.xlabel('Training')
plt.grid(True)
plt.show()

In [29]:
from sklearn.metrics import mean_squared_error, r2_score
# The coefficients
print('Coefficients: \n', lm.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_train, predication_train))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_train, predication_train))

In [30]:
# other way for above command
#plt.scatter(y_train, predication_train, s=30, c='r', marker='+', zorder=10) plt.ylabel("Predicted Values from model") plt.xlabel("Actual Values Prices") plt.show()

#print "MSE:", model.mse_model


In [31]:
predication_test = model.predict(X_test)
# regression coefficients
print('Coefficients: \n', model.coef_)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, predication_test)) 
# variance score: 1 means perfect prediction
print('Variance score: {}'.format(model.score(X_test, y_test)))

In [32]:
## plotting residual errors in test data
plt.scatter(model.predict(X_test), model.predict(X_test) - y_test,
            color = "blue", s = 10, label = 'Test data')
 
## plotting line for zero residual error
plt.hlines(y = 0, xmin = 0, xmax = 50, linewidth = 2)
 
## plotting legend
plt.legend(loc = 'upper right')
 
## plot title
plt.title("Residual errors")
 
## function to show plot
plt.show()

In [33]:
# plot for residual error
 
## setting plot style
plt.style.use('fivethirtyeight')
 
## plotting residual errors in training data
plt.scatter(model.predict(X_train), model.predict(X_train) - y_train,
            color = "green", s = 10, label = 'Train data')
 
## plotting line for zero residual error
plt.hlines(y = 0, xmin = 0, xmax = 50, linewidth = 2)
 
## plotting legend
plt.legend(loc = 'upper right')
 
## plot title
plt.title("Residual errors")
 
## function to show plot
plt.show()

In [34]:
'''K-Folds Cross Validation to improve performace

from sklearn.model_selection import KFold # import KFold kf = KFold(n_splits=2) # Define the split - into 2 folds kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

print(kf)
'''

In [35]:
# Necessary imports: 
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

In [36]:
# Perform 6-fold cross validation
scores = cross_val_score(model, X_train, y_train, cv=6)
print ('Cross-validated scores:', scores)
max(scores[np.argsort(scores)])

In [37]:
# training

In [38]:
# Make cross validated predictions for training
predictions = cross_val_predict(model, X_train, y_train, cv=2)
plt.scatter(y_train, predictions)
plt.show()

In [39]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_train, predictions))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_train, predictions))

accuracy = metrics.r2_score(y_train, predictions)
print ('Cross-Predicted Accuracy:', accuracy)

In [40]:
# Perform 6-fold cross validation
scores = cross_val_score(model, X_test, y_test, cv=6)
print ('Cross-validated scores:', scores)
max(scores[np.argsort(scores)])

In [41]:
# test

In [42]:
# Make cross validated predictions for test
predictions = cross_val_predict(model, X_test, y_test, cv=2)
plt.scatter(y_test, predictions)
plt.show()

In [43]:
# Make cross validated predictions
predictions = cross_val_predict(model, X_test, y_test, cv=6)
plt.scatter(y_test, predictions)
plt.show()

In [44]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, predictions))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, predictions))

accuracy = metrics.r2_score(y_test, predictions)
print ('Cross-Predicted Accuracy:', accuracy)

In [45]:
#Applying on full data

In [46]:
# Make cross validated predictions
predictions = cross_val_predict(model, x, y, cv=50)
plt.scatter(y, predictions)
plt.show()

In [47]:
print("Mean squared error: %.2f"
      % mean_squared_error(y, predictions))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y, predictions))

accuracy = metrics.r2_score(y, predictions)
print ('Cross-Predicted Accuracy:', accuracy)

In [48]:
# Perform 6-fold cross validation
scores = cross_val_score(model, X_train, y_train, cv=10)
print ('Cross-validated scores:', scores)
max(scores[np.argsort(scores)])

In [49]:
plt.plot(scores)
plt.show()